<a href="https://colab.research.google.com/github/dtuleva/Recipe_Recommendation_System/blob/main/tfrs_multitask_recreate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-task recommenders



In the [basic retrieval tutorial](basic_retrieval) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In addition, building a joint model for multiple tasks may produce better results than building a number of task-specific models. This is especially true where some data is abundant (for example, clicks), and some data is sparse (purchases, returns, manual reviews). In those scenarios, a joint model may be able to use representations learned from the abundant task to improve its predictions on the sparse task via a phenomenon known as [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). For example, [this paper](https://openreview.net/pdf?id=SJxPVcSonN) shows that a model predicting explicit user ratings from sparse user surveys can be substantially improved by adding an auxiliary task that uses abundant click log data.

In this tutorial, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## Imports


Let's first get our imports out of the way.


In [69]:
!pip install -q tensorflow-recommenders


In [70]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf


In [71]:
import tensorflow_recommenders as tfrs

In [72]:
import pandas as pd

## Preparing the dataset

We're going to use the Movielens 100K dataset.

In [73]:
ratings_data = pd.read_csv("/content/mock-data_interaction.csv", index_col = 0)
recipes_data = pd.read_csv("/content/mock-data_recipe.csv", index_col = 0)

In [74]:
recipes_data.shape
ratings_data.shape

(8671, 4)

In [75]:
# ratings = ratings_data[ratings_data.rating > 3] # assume positive rating over 3
# ratings.shape

In [76]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_data))
recipes = tf.data.Dataset.from_tensor_slices(dict(recipes_data))

In [77]:
# ratings = tfds.load('movielens/100k-ratings', split="train")
# movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"],
    "rating": x["rating"],
})
recipes = recipes.map(lambda x: x["recipe_id"])

And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [78]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(ratings.cardinality(), seed=42, reshuffle_each_iteration=False)

train = shuffled.take(6_000)
test = shuffled.skip(6_000).take(2_000)

recipe_ids = recipes.batch(100)
user_ids = ratings.batch(5000).map(lambda x: x["user_id"])

unique_recipe_ids = np.unique(np.concatenate(list(recipe_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [79]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(ratings.cardinality(), seed=42, reshuffle_each_iteration=False)

ratings_train = shuffled.take(6_000)
ratings_test = shuffled.skip(6_000).take(2_000)

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this tutorial, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

```python
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])
```

However, now we will have two tasks. The first is the rating task:

```python
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)
```

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

```python
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)
```

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [80]:
class RecipeRecommendModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.recipe_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.IntegerLookup(
        vocabulary=unique_recipe_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.IntegerLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=recipes.batch(100).map(self.recipe_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.recipe_model(features["recipe_id"])

    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [81]:
model = RecipeRecommendModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [82]:
#model.user_model([666])

In [83]:
# model.summary(expand_nested = True)

In [84]:
cached_train = train.shuffle(train.cardinality()).batch(124).cache()
cached_test = test.batch(124).cache()

In [85]:
# for el in cached_train.take(1):
#   print(el)
#   print(el['recipe_id'].shape)
#   print(el.keys())

In [86]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
49/49 [==============================] - 10s 154ms/step - root_mean_squared_error: 1.3587 - factorized_top_k/top_1_categorical_accuracy: 0.0085 - factorized_top_k/top_5_categorical_accuracy: 0.0645 - factorized_top_k/top_10_categorical_accuracy: 0.1198 - factorized_top_k/top_50_categorical_accuracy: 0.4965 - factorized_top_k/top_100_categorical_accuracy: 0.9917 - loss: 1.8032 - regularization_loss: 0.0000e+00 - total_loss: 1.8032
Epoch 2/3
49/49 [==============================] - 12s 246ms/step - root_mean_squared_error: 0.7954 - factorized_top_k/top_1_categorical_accuracy: 0.0115 - factorized_top_k/top_5_categorical_accuracy: 0.0678 - factorized_top_k/top_10_categorical_accuracy: 0.1208 - factorized_top_k/top_50_categorical_accuracy: 0.4988 - factorized_top_k/top_100_categorical_accuracy: 0.9923 - loss: 0.6269 - regularization_loss: 0.0000e+00 - total_loss: 0.6269
Epoch 3/3
17/17 [==============================] - 2s 66ms/step - root_mean_squared_error: 0.8557 - factorized_t

The model does OK on predicting ratings (with an RMSE of around 1.11), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [87]:
model = RecipeRecommendModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [88]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
49/49 [==============================] - 7s 116ms/step - root_mean_squared_error: 4.6211 - factorized_top_k/top_1_categorical_accuracy: 0.0058 - factorized_top_k/top_5_categorical_accuracy: 0.1250 - factorized_top_k/top_10_categorical_accuracy: 0.1933 - factorized_top_k/top_50_categorical_accuracy: 0.4925 - factorized_top_k/top_100_categorical_accuracy: 0.9507 - loss: 581.2761 - regularization_loss: 0.0000e+00 - total_loss: 581.2761
Epoch 2/3
49/49 [==============================] - 6s 122ms/step - root_mean_squared_error: 4.7122 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.8288 - factorized_top_k/top_10_categorical_accuracy: 0.9047 - factorized_top_k/top_50_categorical_accuracy: 0.9843 - factorized_top_k/top_100_categorical_accuracy: 0.9993 - loss: 552.3627 - regularization_loss: 0.0000e+00 - total_loss: 552.3627
Epoch 3/3
17/17 [==============================] - 1s 61ms/step - root_mean_squared_error: 4.9012 - fa

We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [89]:
model = RecipeRecommendModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [90]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
49/49 [==============================] - 6s 77ms/step - root_mean_squared_error: 1.4346 - factorized_top_k/top_1_categorical_accuracy: 0.0028 - factorized_top_k/top_5_categorical_accuracy: 0.1140 - factorized_top_k/top_10_categorical_accuracy: 0.1923 - factorized_top_k/top_50_categorical_accuracy: 0.4857 - factorized_top_k/top_100_categorical_accuracy: 0.9603 - loss: 583.2635 - regularization_loss: 0.0000e+00 - total_loss: 583.2635
Epoch 2/3
49/49 [==============================] - 4s 81ms/step - root_mean_squared_error: 0.8963 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.8148 - factorized_top_k/top_10_categorical_accuracy: 0.8955 - factorized_top_k/top_50_categorical_accuracy: 0.9805 - factorized_top_k/top_100_categorical_accuracy: 0.9997 - loss: 559.3485 - regularization_loss: 0.0000e+00 - total_loss: 559.3485
Epoch 3/3
17/17 [==============================] - 2s 63ms/step - root_mean_squared_error: 0.8521 - fact

The result is a model that performs roughly as well on both tasks as each specialized model.

### Making prediction

We can use the trained multitask model to get trained user and movie embeddings, as well as the predicted rating:

In [91]:
test_users = ratings_data.user_id[:3].tolist()
test_users

[8542392, 11174581, 8262477]

In [92]:
test_recipes = ratings_data.recipe_id[:3].tolist()
test_recipes

[222388, 222388, 222388]

In [93]:
test_ratings = ratings_data.rating[:3].tolist()
test_ratings # ground truth

[5, 5, 5]

In [94]:
trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
      "user_id": np.array(test_users),
      "recipe_id": np.array(test_recipes)
  })
print("Predicted rating:")
print(predicted_rating)

Predicted rating:
tf.Tensor(
[[4.302632 ]
 [4.24997  ]
 [4.2820783]], shape=(3, 1), dtype=float32)


While the results here do not show a clear accuracy benefit from a joint model in this case, multi-task learning is in general an extremely useful tool. We can expect better results when we can transfer knowledge from a data-abundant task (such as clicks) to a closely related data-sparse task (such as purchases).